In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path  = "valid-data.tsv"

In [ ]:
names = ["class", "message"]

In [ ]:
df_train = pd.read_csv(train_file_path, sep="\t", names=names)
df_train.head()

In [ ]:
df_test = pd.read_csv(test_file_path, sep="\t", names=names)
df_test.head()

In [ ]:
print(len(df_train))
print(len(df_test))

# 1. Handle Data and Exploring Data

In [ ]:
y_train = np.array([0 if x=="ham" else 1 for x in df_train['class']])
y_test  = np.array([0 if x=="ham" else 1 for x in df_test['class']])

In [ ]:
bar = df_train['class'].value_counts()

plt.bar(bar.index, bar)
plt.xlabel('Class')
plt.title('Number of ham and spam messages')

# 2. Text processing

In [ ]:
import nltk
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from nltk.stem import PorterStemmer, WordNetLemmatizer

In [ ]:
nltk.download('punkt')
nltk.download('stopwords') # download stopwords
nltk.download('wordnet')   # download vocab for lemmatizer

In [ ]:
lemmatizer = WordNetLemmatizer()
def preprocess_text(text):
    # Converts text to lowercase
    text = text.lower()

    # Remove punctuation
    text = "".join([char for char in text if char not in punctuation])

    # Removes stop words (common, uninformative words like "the", "a")
    stop_words = stopwords.words('english')
    text = " ".join([word for word in text.split() if word not in stop_words])

    # Stemming
    stemmer = PorterStemmer()
    text = " ".join([stemmer.stem(word) for word in text.split()])

    # Lemmatizes text using WordNetLemmatizer
    text = re.sub(r'([^\s\w])+', ' ', text)
    text = " ".join([lemmatizer.lemmatize(word) for word in text.split()
                    if not word in stop_words])

    # words = text.split()
    # pos_tags = pos_tag(words)
    # lemmatized_words = [lemmatizer.lemmatize(word, pos='n' if pos.startswith('N') else pos) for word, pos in pos_tags]
    # text = " ".join(lemmatized_words)

    return text

In [ ]:
X_train = df_train['message'].apply(lambda x: preprocess_text(x))
X_train[:5]

X_test = df_test['message'].apply(lambda x: preprocess_text(x))
X_test[:5]

# 3. Tokenization

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from sklearn.model_selection import train_test_split

In [ ]:
# Define tokenization parameters
# Consider adjusting based on data size and complexity
num_words = 1000

# Cut off the words after seeing 500 words in each document
max_len = 500

# Create and fit tokenizer
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(X_train)

In [ ]:
# Transform each text to a sequence of integers
sequences = tokenizer.texts_to_sequences(X_train)
sequences[:5]

In [ ]:
# Make all rows of equal length
X_train_vec = sequence.pad_sequences(sequences, maxlen=max_len)
X_train_vec[:5]

# 4. Create Model

In [ ]:
i = tf.keras.layers.Input(shape=[max_len])
x = tf.keras.layers.Embedding(num_words, 50)(i)
x = tf.keras.layers.LSTM(64)(x) # Adjust hidden units

x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(1, activation='sigmoid')(x) # Output layer for binary classification

model = tf.keras.models.Model(inputs=i, outputs=x)
model.compile(loss='binary_crossentropy',optimizer='RMSprop',metrics=['accuracy'])
model.summary()

In [ ]:
model_fit = model.fit(X_train_vec, y_train,
              batch_size=128, epochs=10,
              validation_split=0.2,
              callbacks=[tf.keras.callbacks.EarlyStopping(
              monitor='val_loss', min_delta=0.0001)])

# 5. Evaluate and Improve (Optional):

In [ ]:
plt.plot(model_fit.history['loss'], label='loss')
plt.plot(model_fit.history['val_loss'], label='val_loss')
plt.legend()

In [ ]:
plt.plot(model_fit.history['accuracy'], label='acc')
plt.plot(model_fit.history['val_accuracy'], label='val_acc')
plt.legend()

In [ ]:
def preprocessing(X):
  x = X.apply(lambda x: preprocess_text(x))
  x = tokenizer.texts_to_sequences(x)
  return sequence.pad_sequences(x, maxlen=max_len)

In [ ]:
s = model.evaluate(preprocessing(df_test['message']), y_test)

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  p = model.predict(preprocessing(pd.Series([pred_text])))[0]

  return (p[0], ("ham" if p<0.5 else "spam"))

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
